In [61]:
%reset -f
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
import sympy as sy
import scipy

mass_other = 667800 #kg
rocket_diameter = 10.1 #m
ideal_velocity_change = 3600 #m/s
specific_impulse = 283 #s
oxidizer_to_fuel_mass_ratio = 2.27
one_engine_mass = 8400 #kg
number_engines = 5
RP1_density = 810 #kg/m^3
LOX_density = 1141 #kg/m^3
tank_thickness = 5e-3 #m
rocket_body_thickness = 2e-2 #m
density_aluminum = 2810 #kg/m^3
g_sl = 9.81
C = specific_impulse * g_sl
MR = np.exp(ideal_velocity_change/C)

#guess for propellant mass
initial_propellant_mass = 2.25e6

mass_initial = initial_propellant_mass / (1 - (1/(MR)))

length = sy.Symbol('l')
length_fuel = (length - 10) / (1 + oxidizer_to_fuel_mass_ratio * (RP1_density / LOX_density))
length_oxide = length - length_fuel - 10

outer_diameter_rocket = rocket_diameter
inner_diameter_rocket = outer_diameter_rocket - 2 * rocket_body_thickness
outer_diameter_tank = inner_diameter_rocket
inner_diameter_tank = outer_diameter_tank - 2 * tank_thickness

mass_rocket = (np.pi / 4) * (outer_diameter_rocket**2 - inner_diameter_rocket**2) * (length + 10 + 6) * density_aluminum + 2 * (np.pi / 4) * outer_diameter_rocket**2 * rocket_body_thickness * density_aluminum
mass_fuel_tank = (np.pi / 4) * (outer_diameter_tank**2 - inner_diameter_tank**2) * (length_fuel + 3) * density_aluminum + 2 * (np.pi / 4) * outer_diameter_tank**2 * tank_thickness * density_aluminum
mass_oxide_tank = (np.pi / 4) * (outer_diameter_tank**2 - inner_diameter_tank**2) * (length_oxide + 3) * density_aluminum + 2 * (np.pi / 4) * outer_diameter_tank**2 * tank_thickness * density_aluminum
mass_fuel = (np.pi / 4) * (inner_diameter_tank)**2 * (length_fuel) * RP1_density
mass_oxide = (np.pi / 4) * (inner_diameter_tank)**2 * (length_oxide) * LOX_density
mass_engines = number_engines * one_engine_mass
total_mass_expr = mass_engines + mass_rocket + mass_fuel + mass_oxide + mass_fuel_tank + mass_oxide_tank + mass_other

#solve for length
equation = sy.Eq(mass_initial, total_mass_expr)
length_solve = sy.solve(equation, length)[0]
length_fuel_solve = length_fuel.subs(length,length_solve)
length_oxide_solve = length_oxide.subs(length,length_solve)
mass_oxide_solve = mass_oxide.subs(length,length_solve)
mass_fuel_solve = mass_fuel.subs(length,length_solve)
mass_propellant_solve = mass_oxide_solve + mass_fuel_solve
dry_mass_solve = mass_rocket.subs(length,length_solve) + mass_fuel_tank.subs(length,length_solve) + mass_oxide_tank.subs(length,length_solve)



err = (mass_propellant_solve - initial_propellant_mass) / initial_propellant_mass

print(f"propellant mass: {mass_propellant_solve} kg")
print(f"rocket body length: {length_solve + 6} m")
print(f"fuel tank length: {length_fuel_solve + 3} m")
print(f"oxide tank length: {length_oxide_solve + 3} m")
print(f"dry mass (structural mass not counting engines): {dry_mass_solve} kg")
print(f"relative error of calculated propellant mass to initial guess: {err * 100}%")







The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
propellant mass: 2262007.12273244 kg
rocket body length: 44.1142231901167 m
fuel tank length: 13.7656236063775 m
oxide tank length: 20.3485995837392 m
dry mass (structural mass not counting engines): 124920.068180132 kg
relative error of calculated propellant mass to initial guess: 0.533649899219618%
